In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!pip install requests folium
!pip install openrouteservice

In [ ]:
import pandas as pd

df_clients = pd.read_csv('./content/caso2/clients.csv')
df_depots = pd.read_csv('./content/caso2/depots.csv')
df_vehicles = pd.read_csv('./content/caso2/vehicles.csv')

In [ ]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [ ]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C


print("model.D =", list(model.D.data()))
print("model.C =", list(model.C.data()))
print("model.V =", list(model.V.data()))
print("model.N =", list(model.N.data()))
print((lugares))
print(len(lugares))

model.D = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12']
model.C = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
model.V = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6']
model.N = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5', 'CD6', 'CD7', 'CD8', 'CD9', 'CD10', 'CD11', 'CD12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
[(-74.08124218159384, 4.75021190869025), (-74.10993358606953, 4.5363832206427785), (-74.03854814565923, 4.792925960208614), (-74.06706883098641, 4.72167778077445), (-74.13826337931849, 4.607707046760958), (-74.12400186370824, 4.650463053612691), (-74.09561875464892, 4.621911772492814), (-74.10975623736951, 4.678960680833056), (-74.09547235719887, 4.735973062153282), (-74.10991610076434, 4.550640992537941), (-74.10977422186991, 4.664702960902753), (-74.12408925943333, 4.5791740634103135), (-74.19699184741948, 4.632552840424734), (-74.15503659129905, 4.601328214582278), (-74.10178731827096, 4.732421040989568), (-74.19486224157397, 4

In [ ]:
import openrouteservice
from openrouteservice import convert
import json

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'

client = openrouteservice.Client(key=API_KEY)

matrix = client.distance_matrix(
    locations=lugares,
    profile='driving-car',
    metrics=['distance'],
    resolve_locations=True,
    units='km',
)

In [ ]:
precio_km = 3032.1
cost = {}
distance = {}

dist_matrix = matrix['distances']
print(dist_matrix)
print(len(dist_matrix))

for i,i1 in zip(range(len(lugares)),  list(model.N.data())):
        for j,j1 in zip(range(len(lugares)),  list(model.N.data())):
            if i != j:
                distancia_km = dist_matrix[i][j]
                precio = round(distancia_km * precio_km, 3)
            else:
                distancia_km = 9999999
                precio =9999999
            distance[(i1, j1)] = distancia_km
            cost[(i1, j1)] = precio
print(distance)
print(cost)


[[0.0, 32.99, 10.7, 7.49, 20.95, 17.04, 21.1, 13.71, 4.33, 30.68, 13.63, 26.3, 22.49, 25.99, 4.27, 22.43, 12.13, 18.17, 19.64, 7.73, 17.69], [33.54, 0.0, 34.08, 29.96, 13.84, 19.16, 14.14, 24.19, 32.07, 2.58, 22.37, 6.42, 20.41, 15.27, 33.01, 22.06, 31.4, 24.56, 21.02, 29.68, 16.57], [13.9, 38.88, 0.0, 15.76, 32.09, 28.18, 26.98, 24.86, 17.63, 36.57, 24.77, 32.18, 36.07, 34.69, 17.99, 36.01, 25.71, 32.24, 23.5, 16.04, 28.83], [5.5, 29.42, 10.61, 0.0, 17.38, 13.47, 17.53, 10.14, 7.05, 27.11, 10.06, 22.72, 21.36, 22.42, 7.99, 21.29, 10.99, 17.53, 16.07, 3.05, 14.11], [20.63, 15.06, 27.62, 17.06, 0.0, 6.26, 8.53, 11.28, 19.17, 9.56, 10.94, 4.87, 11.71, 2.7, 20.1, 11.64, 18.5, 11.66, 21.04, 16.77, 8.91], [14.73, 18.66, 21.72, 11.15, 6.78, 0.0, 5.96, 5.38, 13.26, 16.35, 4.49, 10.5, 10.77, 9.86, 14.19, 10.71, 12.59, 6.48, 16.14, 10.87, 4.7], [19.64, 13.82, 22.64, 16.06, 9.38, 5.86, 0.0, 10.8, 18.18, 11.51, 7.03, 7.21, 14.23, 9.73, 19.11, 14.17, 18.18, 10.96, 12.59, 15.78, 3.26], [13.91, 23.3

In [ ]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)


{'CD1': 8, 'CD2': 10, 'CD3': 0, 'CD4': 4, 'CD5': 28, 'CD6': 3, 'CD7': 0, 'CD8': 10, 'CD9': 43, 'CD10': 1, 'CD11': 16, 'CD12': 18}
{'C1': 12, 'C2': 15, 'C3': 15, 'C4': 6, 'C5': 5, 'C6': 11, 'C7': 12, 'C8': 10, 'C9': 15}
{'V1': 131.9211396722696, 'V2': 108.4356199315333, 'V3': 91.50425520531196, 'V4': 32.896064077536955, 'V5': 22.65262807032524, 'V6': 22.682911535937688}
{'V1': 145.85207096486445, 'V2': 1304.605971281605, 'V3': 953.172608610164, 'V4': 17.302304187458727, 'V5': 16.627680130757895, 'V6': 13.602810739289229}


In [ ]:
model.cost = Param(model.N, model.N, initialize=lambda model, i, j: cost[(i, j)], within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda model, i, j: distance[(i, j)], within=NonNegativeReals)

# Variables de decisión y auxiliares

model.x = Var(model.V, model.N, model.N, domain=Binary)
model.y = Var(model.V, model.D, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# funcion objetivo

def obj_rule(model):
    return sum(model.cost[i,j] * model.x[v,i,j] for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

def assign_depot_rule(model, v):
    return sum(model.y[v,d] for d in model.D) == 1

model.assign_depot = Constraint(model.V, rule=assign_depot_rule)

def start_route_rule(model, v, d):
    return sum(model.x[v, d, j] for j in model.N if j != d) == model.y[v,d]

model.start_route = Constraint(model.V, model.D, rule=start_route_rule)


def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    else:
        return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

def no_depot_to_depot_rule(model, v, i, j):
    if i in model.D and j in model.D and i != j:
        return model.x[v,i,j] == 0
    else:
        return Constraint.Skip

model.no_depot_travel = Constraint(model.V, model.N, model.N, rule=no_depot_to_depot_rule)

def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    else:
        return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

def depot_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= sum(model.y[v,d] * depot_capacity[d] for d in model.D)

model.depot_capacity = Constraint(model.V, rule=depot_capacity_rule)

def vehicle_range_rule(model, v):
    return sum(model.distance[i,j] * model.x[v,i,j] for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)

def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v,i] - model.u[v,j] + vehicle_capacity[v] * model.x[v,i,j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v,i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v,i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

from amplpy import modules

#solver = SolverFactory('glpk')
# resolver
solver_name = "gurobi"
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
results = solver.solve(model)

In [ ]:
from pyomo.environ import value

def print_routes(model):
    print(f"\n==== Costo óptimo total del plan de rutas: $ {value(model.obj):.3f}")
    for v in model.V:
        print(f"\n>>> Ruta del vehículo {v}:")

        # 1. Encontrar el depósito asignado al vehículo y su capacidad
        depot = None
        for d in model.D:
            if value(model.y[v, d]) > 0.5:
                depot = d
                break

        if depot is None:
            print("Vehículo no asignado a ningún depósito.")
            continue

        print(f" - Depósito de salida: {depot} (Capacidad depósito: {depot_capacity[depot]})")
        print(f" - Capacidad del vehículo: {vehicle_capacity[v]} unidades")
        print(f" - Rango operativo del vehículo: {vehicle_range[v]} km")

        # 2. Reconstruir la ruta
        current_node = depot
        route = [current_node]
        visited = set()
        deliveries = {}

        while True:
            next_node = None
            for j in model.N:
                if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                    next_node = j
                    break

            if next_node is None or next_node in visited:
                break

            if next_node in model.C:
                deliveries[(current_node, next_node)] = demand[next_node]

            route.append(next_node)
            visited.add(next_node)
            current_node = next_node

        # 3. Calcular carga total y distancia total
        total_load = sum(deliveries.values())
        total_distance = sum(
            value(model.distance[route[i], route[i+1]])
            for i in range(len(route)-1)
        )

        # 4. Imprimir resultados
        if len(route) > 1:
            print(f" - Ruta: {' → '.join(route)}")
            print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]} unidades")
            print(f" - Distancia total recorrida: {total_distance:.3f} km")
            porcentaje = total_distance / vehicle_range[v] * 100
            print(f" - Uso del rango operativo: {porcentaje:.1f}%")
            print(f' - Costo del transporte: $ {total_distance*precio_km:.2f}')

            print(" - Entregas entre nodos:")
            for (i, j), qty in deliveries.items():
                print(f"    De {i} → {j}: {qty} unidades")
        else:
            print("Vehículo no utilizado.")
print_routes(model)


==== Costo óptimo total del plan de rutas: $ 284744.511

>>> Ruta del vehículo V1:
 - Depósito de salida: CD9 (Capacidad depósito: 43)
 - Capacidad del vehículo: 131.9211396722696 unidades
 - Rango operativo del vehículo: 145.85207096486445 km
 - Ruta: CD9 → C5 → CD9
 - Carga total transportada: 5/131.9211396722696 unidades
 - Distancia total recorrida: 17.540 km
 - Uso del rango operativo: 12.0%
 - Costo del transporte: $ 53183.03
 - Entregas entre nodos:
    De CD9 → C5: 5 unidades

>>> Ruta del vehículo V2:
 - Depósito de salida: CD9 (Capacidad depósito: 43)
 - Capacidad del vehículo: 108.4356199315333 unidades
 - Rango operativo del vehículo: 1304.605971281605 km
 - Ruta: CD9 → C8 → C7 → C9 → CD7
 - Carga total transportada: 37/108.4356199315333 unidades
 - Distancia total recorrida: 30.460 km
 - Uso del rango operativo: 2.3%
 - Costo del transporte: $ 92357.77
 - Entregas entre nodos:
    De CD9 → C8: 10 unidades
    De C8 → C7: 12 unidades
    De C7 → C9: 15 unidades

>>> Ruta d

In [ ]:
!pip install openrouteservice folium

In [ ]:
import folium
import openrouteservice
from openrouteservice import convert
from pyomo.environ import value
import itertools

# Paleta de colores (puedes expandirla si tienes más vehículos)
colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'black', 'pink', 'gray', 'cyan']
vehiculo_a_color = dict(zip(model.V, itertools.cycle(colores)))


client = openrouteservice.Client(key=API_KEY)

m = folium.Map(location=[4.65, -74.1], zoom_start=11)

coords= lugares
labels= list(model.N.data())

In [ ]:
for label, coord in zip(labels, coords):
    folium.Marker(location=(coord[1], coord[0]), popup=label).add_to(m)

# Diccionario de índice a coordenada
label_to_coord = dict(zip(labels, coords))

# Supón que tienes model.x[v, i, j] y model.V y model.N cargado
# Simulación: rutas como lista de tuplas (vehículo, origen, destino)
rutas_optimas = []
for v in model.V:
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                rutas_optimas.append((v, i, j))
print(rutas_optimas)



[('V1', 'CD9', 'C5'), ('V1', 'C5', 'CD9'), ('V2', 'CD9', 'C8'), ('V2', 'C7', 'C9'), ('V2', 'C8', 'C7'), ('V2', 'C9', 'CD7'), ('V3', 'CD5', 'C4'), ('V3', 'C1', 'CD5'), ('V3', 'C4', 'C1'), ('V4', 'CD9', 'C3'), ('V4', 'C3', 'CD9'), ('V5', 'CD11', 'C6'), ('V5', 'C6', 'CD6'), ('V6', 'CD5', 'C2'), ('V6', 'C2', 'CD5')]


In [ ]:
for (v, i, j) in rutas_optimas:
    coord_i = label_to_coord[i]
    coord_j = label_to_coord[j]
    color = vehiculo_a_color[v]
    try:
        route = client.directions(
            coordinates=[coord_i, coord_j],
            profile='driving-car',
            format='geojson'
        )
        folium.GeoJson(
            route,
            name=f"Ruta {v}: {i}->{j}",
            tooltip=f"{v}: {i} → {j}",
            style_function=lambda x, col=color: {'color': col, 'weight': 4, 'opacity': 0.8}
        ).add_to(m)
    except Exception as e:
        print(f"Error en ruta {i} → {j}: {e}")
m

In [2]:
import re
import string

# Texto de entrada (sustituye "..." por el texto completo que copiaste del plan de rutas)
raw_text = """==== Costo óptimo total del plan de rutas: $ 284744.511

>>> Ruta del vehículo V1:
 - Depósito de salida: CD9 (Capacidad depósito: 43)
 - Capacidad del vehículo: 131.9211396722696 unidades
 - Rango operativo del vehículo: 145.85207096486445 km
 - Ruta: CD9 → C5 → CD9
 - Carga total transportada: 5/131.9211396722696 unidades
 - Distancia total recorrida: 17.540 km
 - Uso del rango operativo: 12.0%
 - Costo del transporte: $ 53183.03
 - Entregas entre nodos:
    De CD9 → C5: 5 unidades

>>> Ruta del vehículo V2:
 - Depósito de salida: CD9 (Capacidad depósito: 43)
 - Capacidad del vehículo: 108.4356199315333 unidades
 - Rango operativo del vehículo: 1304.605971281605 km
 - Ruta: CD9 → C8 → C7 → C9 → CD7
 - Carga total transportada: 37/108.4356199315333 unidades
 - Distancia total recorrida: 30.460 km
 - Uso del rango operativo: 2.3%
 - Costo del transporte: $ 92357.77
 - Entregas entre nodos:
    De CD9 → C8: 10 unidades
    De C8 → C7: 12 unidades
    De C7 → C9: 15 unidades

>>> Ruta del vehículo V3:
 - Depósito de salida: CD5 (Capacidad depósito: 28)
 - Capacidad del vehículo: 91.50425520531196 unidades
 - Rango operativo del vehículo: 953.172608610164 km
 - Ruta: CD5 → C4 → C1 → CD5
 - Carga total transportada: 18/91.50425520531196 unidades
 - Distancia total recorrida: 22.730 km
 - Uso del rango operativo: 2.4%
 - Costo del transporte: $ 68919.63
 - Entregas entre nodos:
    De CD5 → C4: 6 unidades
    De C4 → C1: 12 unidades

>>> Ruta del vehículo V4:
 - Depósito de salida: CD9 (Capacidad depósito: 43)
 - Capacidad del vehículo: 32.896064077536955 unidades
 - Rango operativo del vehículo: 17.302304187458727 km
 - Ruta: CD9 → C3 → CD9
 - Carga total transportada: 15/32.896064077536955 unidades
 - Distancia total recorrida: 2.560 km
 - Uso del rango operativo: 14.8%
 - Costo del transporte: $ 7762.18
 - Entregas entre nodos:
    De CD9 → C3: 15 unidades

>>> Ruta del vehículo V5:
 - Depósito de salida: CD11 (Capacidad depósito: 16)
 - Capacidad del vehículo: 22.65262807032524 unidades
 - Rango operativo del vehículo: 16.627680130757895 km
 - Ruta: CD11 → C6 → CD6
 - Carga total transportada: 11/22.65262807032524 unidades
 - Distancia total recorrida: 15.070 km
 - Uso del rango operativo: 90.6%
 - Costo del transporte: $ 45693.75
 - Entregas entre nodos:
    De CD11 → C6: 11 unidades

>>> Ruta del vehículo V6:
 - Depósito de salida: CD5 (Capacidad depósito: 28)
 - Capacidad del vehículo: 22.682911535937688 unidades
 - Rango operativo del vehículo: 13.602810739289229 km
 - Ruta: CD5 → C2 → CD5
 - Carga total transportada: 15/22.682911535937688 unidades
 - Distancia total recorrida: 5.550 km
 - Uso del rango operativo: 40.8%
 - Costo del transporte: $ 16828.16
 - Entregas entre nodos:
    De CD5 → C2: 15 unidades"""

vehicles_data = []

vehicle_blocks = raw_text.strip().split(">>> Ruta del vehículo ")
for block in vehicle_blocks[1:]:
    lines = block.strip().split('\n')
    vehicle_number = re.search(r"V(\d+):", lines[0]).group(1)
    vehicle_id = f"VEH{int(vehicle_number):03d}"

    # Detectar número de depósito y mapear a letra (CD1 → A → CDA)
    depot_match = re.search(r"Depósito de salida: CD(\d+)", block)
    depot_num = int(depot_match.group(1))
    depot_id = f"CD{depot_num}"
    depot_letter = string.ascii_uppercase[depot_num - 1]
    depot_code = f"CD{depot_num}"
    depot_id_formatted = f"CD{depot_letter}"

    cap = int(re.search(r"Capacidad del vehículo: (\d+)", block).group(1))
    route = re.search(r"Ruta: (.+)", block).group(1).strip()
    route_ids = re.findall(r"C\d+|CD\d+", route)

    # Convertir ruta
    def format_node(x):
        if x.startswith("CD"):
            num = int(x[2:])
            return f"CD{string.ascii_uppercase[num - 1]}"
        else:
            return f"C{x[1:].zfill(3)}"

    route_sequence = '-'.join([format_node(x) for x in route_ids])
    clients = [x for x in route_ids if not x.startswith("CD")]
    client_count = len(clients)

    demands_raw = re.findall(r"De [^:]+: (\d+) unidades", block)
    demands = '-'.join(demands_raw)

    distance = re.search(r"Distancia total recorrida: ([\d.]+)", block).group(1)
    cost = re.search(r"Costo del transporte: \$ ([\d.]+)", block).group(1)

    vehicles_data.append([
        vehicle_id, depot_id_formatted, cap, route_sequence, client_count,
        demands, distance, 0, cost
    ])

# Generar contenido
header = "VehicleId,DepotId,InitialLoad,RouteSequence,ClientsServed,DemandsSatisfied,TotalDistance,TotalTime,FuelCost"
lines = [header] + [','.join(map(str, row)) for row in vehicles_data]
output_text = '\n'.join(lines)

# Guardar archivo
with open("respuesta.txt", "w", encoding="utf-8") as f:
    f.write(output_text)

# Mostrar en consola
print(output_text)

# Descargar en Colab
from google.colab import files
files.download("respuesta.txt")

VehicleId,DepotId,InitialLoad,RouteSequence,ClientsServed,DemandsSatisfied,TotalDistance,TotalTime,FuelCost
VEH001,CDI,131,CDI-C005-CDI,1,5,17.540,0,53183.03
VEH002,CDI,108,CDI-C008-C007-C009-CDG,3,10-12-15,30.460,0,92357.77
VEH003,CDE,91,CDE-C004-C001-CDE,2,6-12,22.730,0,68919.63
VEH004,CDI,32,CDI-C003-CDI,1,15,2.560,0,7762.18
VEH005,CDK,22,CDK-C006-CDF,1,11,15.070,0,45693.75
VEH006,CDE,22,CDE-C002-CDE,1,15,5.550,0,16828.16


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>